Download libraries

In [374]:
import sys
import os
import time

In [375]:
import pandas as pd
import numpy as np

In [376]:
from sklearn.cross_validation import train_test_split
import numpy as np

In [377]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import classification_report

Set working directory

In [378]:
print os.getcwd();

/Users/imacair/Desktop/Products3


In [379]:
os.chdir("/Users/imacair/Desktop/Products3/")

In [380]:
data= pd.read_csv('Final_TW_2104.csv', encoding= "latin-1",delimiter=',',low_memory=False)

In [381]:
#data.dropna(subset=['key']) 

In [447]:
#data['key']

In [415]:
#data[data['key'].str.contains("abbvie")]
data_abbvie = data[data['key'].str.contains('humira', na = False)]

In [416]:
#data = pd.DataFrame({'key': 'abbvie'})

In [417]:
data_abbvie.head()

,Unnamed: 0,Created.At,From.User,From.User.Id,To.User,To.User.Id,Language,Source,Text,Geo.Location.Latitude,Geo.Location.Longitude,Retweet.Count,Id,key,favorited,favoriteCount,isRetweet,retweeted
5389,5390,3/21/2017 5:33,EC,1266364076,NaN,-1,en,<a href='http://twitter.com/download/iphone' r...,Humira: we don't know how it works or if it wo...,NaN,NaN,0,8.44E+17,humira,NaN,NaN,NaN,NaN
5390,5391,3/21/2017 1:27,Seb,1648364930,NaN,-1,en,<a href='http://twitter.com' rel='nofollow'>Tw...,humira doesn't have a twitter. expected yet so...,NaN,NaN,0,8.44E+17,humira,NaN,NaN,NaN,NaN
5391,5392,3/20/2017 22:49,Î_¥ü,7.98E+17,NaN,-1,en,<a href='http://twitter.com/download/iphone' r...,WhT an AMAZING FIRST DAY OF SPRING IT HAS BEEN...,NaN,NaN,0,8.44E+17,humira,NaN,NaN,NaN,NaN
5392,5393,3/20/2017 18:44,kdsak,69589807,NaN,-1,en,<a href='http://twitter.com/download/iphone' r...,Paying full price for my Humira and then havin...,NaN,NaN,0,8.44E+17,humira,NaN,NaN,NaN,NaN
5393,5394,3/20/2017 18:11,biosimilars101,8.11E+17,NaN,-1,en,<a href='https://ads.twitter.com' rel='nofollo...,#DYK: AbbVie typically rolls out two price hik...,NaN,NaN,1,8.44E+17,humira,NaN,NaN,NaN,NaN


Reads the file

In [418]:
#data = pd.read_csv('trainingandtestdata/training.1600000.processed.noemoticon.csv',
#                   names =['sentiment', 'id', 'date', 'query', 'user', 'text'], encoding='latin-1',delimiter=',')
#data

Take a sample from data

In [419]:
sample_data= data.sample(n=10000)
#data_s=sample_data["sentiment"]

In [420]:
data_t=sample_data["Text"]

In [421]:
#data_t.append(Decimal(items['longitude'].encode('utf-16')))

In [422]:
data_t=data_abbvie["Text"]

Preprocess the data

Replacement of abbriviations

In [423]:
replacement = {
'r':'are',
'y':'why',
'u':'you',
'ttyl':'talk to you later',
'l8':'late',
'brb':'be right back',
'lol':'laughing out loud',
'bbl':'be back later',
'tldr':'too long did not read',
'rofl':'rolling on floor laughing',
'gr':'great',
'g2g':'got to go',
'gtg':'got to go',
'cya':'see you',
'cuz':'because',
'cuzz':'because',
'bff':'best friend forever',
'idk':'I do not know',
'sup':'whats up?',
'omg':'oh my gosh',
'nbd':'no big deal',
'tisnf':'this is not fair',
'nw':'no way!',
'rus':'are you serious?',
'myob':'mind your own business',
'm8':'mate',
'gg':'good game',
'gn':'good night',
'hf':'have fun',
'njoy':'enjoy',
'nter':'enter',
'1ce':'once',
'aka':'also known as',
'afk':'away from keyboard',
'jk':'just kidding',
'asap':'as soon as possible',
'app':'application',
'atm':'at the moment',
'b4':'before',
'sux':'sucks',
'fyi':'for your information',
'bday':'birthday',
'k':'okay',
'msg':'message',
'np':'no problem',
'pic':'picture',
'plz':'please',
'sry':'sorry',
'dunno':'do not know',
'wut':'what',
'wuts':'what is',
'ikr':'i know right',
'yolo':'you only live once',
'w/e':'whatever'}

In [424]:
##How in works
s1 = 'y r u l8'

s2 = ' '.join([replacement.get(w, w) for w in s1.split()])
s2

'why are you late'

In [425]:
result = pd.DataFrame()
result = data_t

In [426]:
for i in range(len(result)):
    data_t.values[i]=' '.join([replacement.get(w, w) for w in data_t.values[i].split()])

In [427]:
result.head()

5389    Humira: we don't know how it works or if it wo...
5390    humira doesn't have a twitter. expected yet so...
5391    WhT an AMAZING FIRST DAY OF SPRING IT HAS BEEN...
5392    Paying full price for my Humira and then havin...
5393    #DYK: AbbVie typically rolls out two price hik...
Name: Text, dtype: object

In [428]:
data_t.head()

5389    Humira: we don't know how it works or if it wo...
5390    humira doesn't have a twitter. expected yet so...
5391    WhT an AMAZING FIRST DAY OF SPRING IT HAS BEEN...
5392    Paying full price for my Humira and then havin...
5393    #DYK: AbbVie typically rolls out two price hik...
Name: Text, dtype: object

In [429]:
type(result)

pandas.core.series.Series

In [430]:
data_t=result

In [431]:
#lowercase
data_t = data_t.str.lower()
#Remove hyperlinks
data_t= data_t.str.replace(r'https?:\/\/.*\/[a-zA-Z0-9]*', "")
#Remove citations
data_t = data_t.str.replace(r'@[a-zA-Z0-9]*', "")
#remove punctuation
data_t = data_t.str.replace(r'[^\w\s]',"")
#remove numbers
data_t = data_t.str.replace(r'\d+',"")
#remove _
data_t = data_t.str.replace(r'\_+',"")

In [432]:
data_t.head()

5389    humira we dont know how it works or if it work...
5390    humira doesnt have a twitter expected yet some...
5391    wht an amazing first day of spring it has been...
5392    paying full price for my humira and then havin...
5393    dyk abbvie typically rolls out two price hikes...
Name: Text, dtype: object

In [433]:
df= data_t
df.head()

5389    humira we dont know how it works or if it work...
5390    humira doesnt have a twitter expected yet some...
5391    wht an amazing first day of spring it has been...
5392    paying full price for my humira and then havin...
5393    dyk abbvie typically rolls out two price hikes...
Name: Text, dtype: object

In [434]:
data_t.head()

5389    humira we dont know how it works or if it work...
5390    humira doesnt have a twitter expected yet some...
5391    wht an amazing first day of spring it has been...
5392    paying full price for my humira and then havin...
5393    dyk abbvie typically rolls out two price hikes...
Name: Text, dtype: object

In [435]:
#for i in range(len(data_t)):
 #   print(data_t.values[i])

In [436]:
from textblob import TextBlob

In [437]:
#result = result.iloc[0:].apply(abb)

In [449]:
testimonial = TextBlob(data_t.values[1000])
testimonial.sentiment.polarity
#testimonial.sentiment.subjectivity

0.16666666666666666

In [439]:
res=data_t
res=res.rename(columns = {2:'polarity'})

In [440]:
res2=data_t
res2=res2.rename(columns = {2:'subjectivity'})

In [441]:
for i in range(len(data_t)):
    testimonial = TextBlob(data_t.values[i])
    res.values[i]= testimonial.sentiment.polarity

In [442]:
for i in range(len(data_t)):
    testimonial = TextBlob(data_t.values[i])
    res2.values[i]= testimonial.sentiment.subjectivity

In [443]:
final=pd.concat([data_t, res], axis=1)

In [444]:
final=pd.concat([final, res2], axis=1)

In [445]:
final

,Text,0,0
5389,humira we dont know how it works or if it work...,0,0
5390,humira doesnt have a twitter expected yet some...,-0.35,0.55
5391,wht an amazing first day of spring it has been...,0.0907468,0.658983
5392,paying full price for my humira and then havin...,0.35,0.55
5393,dyk abbvie typically rolls out two price hikes...,-0.166667,0.5
5394,humana is a drug actually no its not its an in...,-0.2,0.35
5395,gotta let it warm up to room temp then its on ...,0.425,0.466667
5396,biologicnaive psa patients persist on therapy ...,0,0
5397,i hit and sedinatation rate and c reactive ...,-0.155556,0.288889
5398,everyday medsminders meds reminder app,-0.2,0.6


In [446]:
#final.to_csv('out.csv', sep=',')

In [32]:
#polarity
final.values[0,1]

-0.2

In [33]:
#subjectivity
final.values[0,2]

0.8